# Projet Data

## Nicolas, Georges et Michael
    

In [1]:
from pandas import read_excel
# Importing necessary library
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus# sample text for performing tokenization
from lxml import html
import requests
from nltk.tokenize import word_tokenize# Passing the string text into word tokenize for breaking the sentences
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import FrenchStemmer
import unidecode
# faut lancer ca la premiere fois -> nltk.download()

## <font color = "darkgreen">1- Chargement des jeux de données</font>

In [2]:
df_data = pd.read_excel("Data/Table_Ciqual_2020_FR_2020_07_07.xls")

## <font color = "darkgreen">2- Analyse la bdd </font>

On va analyser la BDD pour mieux la connaître et par la suite mieux la nettoyer.

In [4]:
#On regarde toute les colonnes du DataFrame
df_data.columns

Index(['alim_grp_code', 'alim_ssgrp_code', 'alim_ssssgrp_code',
       'alim_grp_nom_fr', 'alim_ssgrp_nom_fr', 'alim_ssssgrp_nom_fr',
       'alim_code', 'alim_nom_fr', 'alim_nom_sci',
       'Energie, Règlement UE N° 1169/2011 (kJ/100 g)',
       'Energie, Règlement UE N° 1169/2011 (kcal/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kJ/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kcal/100 g)',
       'Eau (g/100 g)', 'Protéines, N x facteur de Jones (g/100 g)',
       'Protéines, N x 6.25 (g/100 g)', 'Glucides (g/100 g)',
       'Lipides (g/100 g)', 'Sucres (g/100 g)', 'Fructose (g/100 g)',
       'Galactose (g/100 g)', 'Glucose (g/100 g)', 'Lactose (g/100 g)',
       'Maltose (g/100 g)', 'Saccharose (g/100 g)', 'Amidon (g/100 g)',
       'Fibres alimentaires (g/100 g)', 'Polyols totaux (g/100 g)',
       'Cendres (g/100 g)', 'Alcool (g/100 g)', 'Acides organiques (g/100 g)',
       'AG saturés (g/100 g)', 'AG monoinsaturés (g/100 g)',
       'AG polyinsatur

In [5]:
#On garde que les colonne qui nous intéresse pour faire notre 'nutri-score'
df_propre = df_data.copy().drop(['alim_grp_code','alim_ssgrp_code','alim_ssssgrp_code','alim_ssssgrp_nom_fr','alim_ssgrp_code','alim_nom_sci', 
       'Energie, Règlement UE N° 1169/2011 (kJ/100 g)','Energie, N x facteur Jones, avec fibres  (kJ/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kcal/100 g)',
       'Eau (g/100 g)', 'Protéines, N x facteur de Jones (g/100 g)', 'Fructose (g/100 g)',
       'Galactose (g/100 g)', 'Glucose (g/100 g)', 'Lactose (g/100 g)',
       'Maltose (g/100 g)', 'Saccharose (g/100 g)', 'Amidon (g/100 g)',
       'Fibres alimentaires (g/100 g)', 'Polyols totaux (g/100 g)',
       'Cendres (g/100 g)', 'Alcool (g/100 g)', 'Acides organiques (g/100 g)', 'AG monoinsaturés (g/100 g)',
       'AG polyinsaturés (g/100 g)', 'AG 4:0, butyrique (g/100 g)',
       'AG 6:0, caproïque (g/100 g)', 'AG 8:0, caprylique (g/100 g)',
       'AG 10:0, caprique (g/100 g)', 'AG 12:0, laurique (g/100 g)',
       'AG 14:0, myristique (g/100 g)', 'AG 16:0, palmitique (g/100 g)',
       'AG 18:0, stéarique (g/100 g)', 'AG 18:1 9c (n-9), oléique (g/100 g)',
       'AG 18:2 9c,12c (n-6), linoléique (g/100 g)',
       'AG 18:3 c9,c12,c15 (n-3), alpha-linolénique (g/100 g)',
       'AG 20:4 5c,8c,11c,14c (n-6), arachidonique (g/100 g)',
       'AG 20:5 5c,8c,11c,14c,17c (n-3) EPA (g/100 g)',
       'AG 22:6 4c,7c,10c,13c,16c,19c (n-3) DHA (g/100 g)',
       'Cholestérol (mg/100 g)', 'Sel chlorure de sodium (g/100 g)',
       'Calcium (mg/100 g)', 'Chlorure (mg/100 g)', 'Cuivre (mg/100 g)',
       'Fer (mg/100 g)', 'Iode (µg/100 g)', 'Magnésium (mg/100 g)',
       'Manganèse (mg/100 g)', 'Phosphore (mg/100 g)', 'Potassium (mg/100 g)',
       'Sélénium (µg/100 g)', 'Sodium (mg/100 g)', 'Zinc (mg/100 g)',
       'Rétinol (µg/100 g)', 'Beta-Carotène (µg/100 g)',
       'Vitamine D (µg/100 g)', 'Vitamine E (mg/100 g)',
       'Vitamine K1 (µg/100 g)', 'Vitamine K2 (µg/100 g)',
       'Vitamine C (mg/100 g)', 'Vitamine B1 ou Thiamine (mg/100 g)',
       'Vitamine B2 ou Riboflavine (mg/100 g)',
       'Vitamine B3 ou PP ou Niacine (mg/100 g)',
       'Vitamine B5 ou Acide pantothénique (mg/100 g)',
       'Vitamine B6 (mg/100 g)', 'Vitamine B9 ou Folates totaux (µg/100 g)',
       'Vitamine B12 (µg/100 g)'], axis = 1)

In [6]:
#On change les noms des colonnes pour qu'ils soient plus simple
df_propre.columns = ['alim_grp_nom_fr', 'alim_ssgrp_nom_fr', 'alim_code', 'alim_nom_fr', 'Kcal/100 g', 'Protéines/100 g', 'Glucides/100 g', 'Lipides/100 g' ,'Sucres/100 g','AG saturés/100 g']

In [7]:
#On regarde comment les aliments sont écrits dans la colonne pour venir les chercher dans le DataFrame
df_propre.loc[df_propre["alim_nom_fr"].apply(lambda x: "Boeuf" in x)]

,alim_grp_nom_fr,alim_ssgrp_nom_fr,alim_code,alim_nom_fr,Kcal/100 g,Protéines/100 g,Glucides/100 g,Lipides/100 g,Sucres/100 g,AG saturés/100 g
72,entrées et plats composés,plats composés,25033,Boeuf bourguignon,"84,6","8,93","4,17","3,3","1,57","0,88"
104,entrées et plats composés,plats composés,25065,Boeuf aux carottes,"96,6","13,2","3,3","3,15","0,6","1,21"
1145,"viandes, œufs, poissons et assimilés",viandes cuites,6100,"Boeuf, entrecôte, partie maigre, grillée/poêlée",198,"25,5","< 0,1","10,7",0,"4,75"
1146,"viandes, œufs, poissons et assimilés",viandes cuites,6101,"Boeuf, braisé",240,"32,1",0,"12,4",-,"5,72"
1147,"viandes, œufs, poissons et assimilés",viandes cuites,6104,"Boeuf, gîte à la noix, cuit",102,21,"0,2","1,9",-,"0,8"
1148,"viandes, œufs, poissons et assimilés",viandes cuites,6105,"Boeuf, faux-filet, rôti/cuit au four",194,"28,8",traces,"8,71",-,"3,62"
1149,"viandes, œufs, poissons et assimilés",viandes cuites,6110,"Boeuf, faux-filet, grillé/poêlé",182,"27,1",0,"8,17",0,"3,39"
1150,"viandes, œufs, poissons et assimilés",viandes cuites,6123,"Boeuf, plat de côtes, braisé",269,"37,3",0,"13,3",0,"5,58"
1151,"viandes, œufs, poissons et assimilés",viandes cuites,6131,"Boeuf, hampe, grillée/poêlée",174,"21,6",traces,"9,76",-,"4,36"
1152,"viandes, œufs, poissons et assimilés",viandes cuites,6141,"Boeuf, joue, braisée ou bouillie",236,"39,2",traces,"8,82",-,"3,19"


In [8]:
#On regarde tous les groupes d'aliments qu'il y a
df_propre['alim_grp_nom_fr'].value_counts()

viandes, œufs, poissons et assimilés           788
fruits, légumes, légumineuses et oléagineux    614
entrées et plats composés                      337
produits sucrés                                306
produits laitiers et assimilés                 301
eaux et autres boissons                        295
aides culinaires et ingrédients divers         212
produits céréaliers                            145
matières grasses                                75
aliments infantiles                             39
glaces et sorbets                               29
Name: alim_grp_nom_fr, dtype: int64

In [9]:
#On regarde le nombre de 'nan' dans chaque colonne
df_propre.isnull().sum()

alim_grp_nom_fr      45
alim_ssgrp_nom_fr    45
alim_code             0
alim_nom_fr           0
Kcal/100 g           17
Protéines/100 g       1
Glucides/100 g        1
Lipides/100 g         1
Sucres/100 g          9
AG saturés/100 g      5
dtype: int64

In [10]:
#On print la somme des 'nan'
print(np.sum(df_propre.isnull().sum()))

124


In [11]:
#On remarque que 'nan' a 43 occurences du coup nous allons les changer pour traiter la colonne par la suite
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                              303
charcuteries et assimilés                            173
fruits                                               170
viandes crues                                        162
plats composés                                       159
                                                    ... 
denrées destinées à une alimentation particulière      5
céréales et biscuits infantiles                        4
huiles de poissons                                     4
-                                                      1
aides culinaires et ingrédients pour végétariens       1
Name: alim_ssgrp_nom_fr, Length: 64, dtype: int64

## <font color = "darkgreen">3- Nettoyage de la bdd </font>

### <font color = "blue">3.1- Valeur numérique </font>

On va nettoyer la bdd au niveau des valeurs numériques en supprimant les Nan, '-' et '<'.

In [12]:
#On va changer toutes les valeurs 'nan' en 'rien'
for column in df_propre['alim_ssgrp_nom_fr']:
            df_propre['alim_ssgrp_nom_fr'] = df_propre['alim_ssgrp_nom_fr'].replace(to_replace = np.nan, value = 'rien')

In [13]:
#alim_ssgrp_nom_fr est bien à 0
df_propre.isnull().sum()

alim_grp_nom_fr      45
alim_ssgrp_nom_fr     0
alim_code             0
alim_nom_fr           0
Kcal/100 g           17
Protéines/100 g       1
Glucides/100 g        1
Lipides/100 g         1
Sucres/100 g          9
AG saturés/100 g      5
dtype: int64

In [14]:
#On va changer toutes les valeurs '-' en 'rien' dans la colonne alim_ssgrp_nom_fr
for column in df_propre['alim_ssgrp_nom_fr']:
            df_propre['alim_ssgrp_nom_fr'] = df_propre['alim_ssgrp_nom_fr'].replace(['-'], ['rien'])

In [15]:
#On remarque qu'il y a plus de 'nan' et de '-', ils sont tous devenus 'rien'
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                              303
charcuteries et assimilés                            173
fruits                                               170
viandes crues                                        162
plats composés                                       159
                                                    ... 
desserts infantiles                                    5
denrées destinées à une alimentation particulière      5
céréales et biscuits infantiles                        4
huiles de poissons                                     4
aides culinaires et ingrédients pour végétariens       1
Name: alim_ssgrp_nom_fr, Length: 64, dtype: int64

In [16]:
#On supprime les alim_ssgrp_nom_fr qui nous servent pas
sous_groupe_sup = ['céréales et biscuits infantiles', 'rien', 'aides culinaires et ingrédients pour végétariens', 'huiles de poissons',
                  'desserts infantiles', 'denrées destinées à une alimentation particulière', 'aides culinaires et ingrédients pour végétariens',
                  'sorbets', 'barres céréalières', 'glaces', 'desserts glacés', 'petits pots salés et plats infantiles',
                  'autres produits à base de viande', 'laits et boissons infantiles', 'confiseries non chocolatées',
                  'biscuits apéritifs', 'salades composées et crudités', 'viennoiseries', 'feuilletées et autres entrées',
                  'sandwichs', 'pizzas, tartes et crêpes salées', 'gâteaux et pâtisseries', 'plats composés']
i = 0

while(i < len(sous_groupe_sup)):
    df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == sous_groupe_sup[i]].index, inplace = True)
    i = i + 1


In [17]:
#On vérifie bien que les alim_ssgrp_nom_fr sont supprimées
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                              303
charcuteries et assimilés                            173
fruits                                               170
viandes crues                                        162
boissons sans alcool                                 156
produits laitiers frais et assimilés                 137
fromages et assimilés                                135
viandes cuites                                       133
poissons crus                                        106
eaux                                                  90
sauces                                                75
pâtes, riz et céréales                                71
biscuits sucrés                                       65
poissons cuits                                        63
pains et assimilés                                    56
produits à base de poissons et produits de la mer     56
fruits à coque et graines oléagineuses                52
pommes de terre et autres tuber

In [18]:
#Remplace tous les 'traces' en 0 et les '-' en 'Nan' dans les colonnes suivantes
for column in df_propre['Kcal/100 g']:
            df_propre['Kcal/100 g'] = df_propre['Kcal/100 g'].replace(['traces'], ['0'])
            df_propre['Kcal/100 g'] = df_propre['Kcal/100 g'].replace(['-'], value = np.nan, regex = True)            
            
for column in df_propre['Protéines/100 g']:
            df_propre['Protéines/100 g'] = df_propre['Protéines/100 g'].replace(['traces'], ['0'])
            df_propre['Protéines/100 g'] = df_propre['Protéines/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['Glucides/100 g']:
            df_propre['Glucides/100 g'] = df_propre['Glucides/100 g'].replace(['traces'], ['0'])
            df_propre['Glucides/100 g'] = df_propre['Glucides/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['Lipides/100 g']:
            df_propre['Lipides/100 g'] = df_propre['Lipides/100 g'].replace(['traces'], ['0'])
            df_propre['Lipides/100 g'] = df_propre['Lipides/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['Sucres/100 g']:
            df_propre['Sucres/100 g'] = df_propre['Sucres/100 g'].replace(['traces'], ['0'])
            df_propre['Sucres/100 g'] = df_propre['Sucres/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['AG saturés/100 g']:
            df_propre['AG saturés/100 g'] = df_propre['AG saturés/100 g'].replace(['traces'], ['0'])
            df_propre['AG saturés/100 g'] = df_propre['AG saturés/100 g'].replace(['-'], value = np.nan, regex = True)

In [19]:
#On affiche le nombre 'Nan' de chaque colonne
df_propre.isna().sum()

alim_grp_nom_fr        0
alim_ssgrp_nom_fr      0
alim_code              0
alim_nom_fr            0
Kcal/100 g           736
Protéines/100 g       16
Glucides/100 g        79
Lipides/100 g         16
Sucres/100 g         349
AG saturés/100 g     270
dtype: int64

In [20]:
#On compte le nombre de ligne avant de supprimer ceux qui ont une valeur 'Nan'
nombre_total_avant = df_propre['alim_grp_nom_fr'].count()
print(nombre_total_avant)

2605


In [21]:
#On supprime toutes les lignes qui ont une valeur 'Nan'
for column in df_propre:
            df_propre = df_propre.dropna()

In [22]:
#On vérifie
df_propre.isna().sum()

alim_grp_nom_fr      0
alim_ssgrp_nom_fr    0
alim_code            0
alim_nom_fr          0
Kcal/100 g           0
Protéines/100 g      0
Glucides/100 g       0
Lipides/100 g        0
Sucres/100 g         0
AG saturés/100 g     0
dtype: int64

In [23]:
#On recompte le nombre de lignes
nombre_total_apres = df_propre['alim_grp_nom_fr'].count()
print(nombre_total_apres)

1571


In [24]:
#On fait le calcul pour savoir le pourcentage de pertes
print(round(100 - (nombre_total_apres / nombre_total_avant) * 100 , 2), '% de perdu')

39.69 % de perdu


In [25]:
#On regarde les pertes de la colonne 'alim_ssgrp_nom_fr'
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                              132
charcuteries et assimilés                            126
viandes crues                                        124
fromages et assimilés                                105
poissons crus                                         95
produits laitiers frais et assimilés                  85
eaux                                                  69
boissons sans alcool                                  66
poissons cuits                                        61
fruits                                                59
viandes cuites                                        58
pâtes, riz et céréales                                56
pains et assimilés                                    50
pommes de terre et autres tubercules                  47
céréales de petit-déjeuner                            46
soupes                                                45
produits à base de poissons et produits de la mer     45
huiles et graisses végétales   

In [26]:
#On remplace les virgules par des points et on enelve '<' devant certaines valeurs
cols = ['Kcal/100 g', 'Protéines/100 g', 'Glucides/100 g', 'Lipides/100 g', 'Sucres/100 g', 'AG saturés/100 g']

for col in cols:
    df_propre[col] = df_propre[col].map(lambda x: str(x).replace(',', '.').lstrip('<')).astype(float)

Maintentant qu'on a fini de nettoyer les valeurs numériques, on va normaliser les valeurs textuelles.

### <font color = "blue">3.2- Valeur textuelle </font>

In [157]:
french_stopwords = set(stopwords.words('french'))
filtre_stopfr = lambda text: [token for token in text if token.lower() not in french_stopwords]

In [158]:
recipe = []
url = "https://www.marmiton.org/recettes/recette_sauce-bolognaise-ragu-bolognese-recette-officielle_95684.aspx"
page = requests.get(url)
# https://www.marmiton.org/recettes/recette_sauce-puttanesca_14929.aspx
tree = html.fromstring(page.content)
products = tree.xpath("//div[contains(@class, 'MuiGrid-root')]")
for product in products[1:]:
    recipe.append(product.text_content())

In [159]:
recipe

["2\xa0c.à.s d'huile d'olive ",
 '10\xa0cl de vin rouge Sangiovese sec',
 '20\xa0cl de bouillon ',
 '500\xa0g de purée de tomate (passata)',
 "50\xa0g d'oignon haché",
 '50\xa0g de carotte jaune haché',
 ' poivre ',
 ' sel ',
 '500\xa0g de pâtes ',
 '300\xa0g de boeuf maigre haché',
 '100\xa0g de lard doux',
 '50\xa0g de céleri branche haché']

In [211]:
def normalize_text(text, stemmer=True):
    text = unaccented_string = unidecode.unidecode(str(text))    
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    words = filtre_stopfr( word_tokenize(text, language="french") )
    if stemmer:
        stemmer = FrenchStemmer()
        res = " ".join([stemmer.stem(w) for w in words])
    else:
        res = " ".join(words)
    return res

In [161]:
df_propre["alim_nom_fr"] = df_propre["alim_nom_fr"].apply(normalize_text)

In [162]:
df_propre["alim_ssgrp_nom_fr"] = df_propre["alim_ssgrp_nom_fr"].apply(normalize_text)

In [265]:
def word_in_column(df_propre, prod, col):
    mask = df_propre[col].str.contains(prod)
    return df_propre.loc[mask]

In [327]:
def list_match(name):
    l_match = []
    words_test = normalize_text(name).split(" ")
    for word in words_test:
        l_match.append(word_in_column(df_propre, word, 'alim_nom_fr'))
    res = pd.concat(l_match)
    #res.reset_index()["index"].value_counts().iloc[0]
    return l_match

In [328]:
res = list_match("lardon")

In [329]:
res

[                           alim_grp_nom_fr alim_ssgrp_nom_fr  alim_code  \
 1571  viandes, œufs, poissons et assimilés   charcut assimil      28501   
 1579  viandes, œufs, poissons et assimilés   charcut assimil      28720   
 1580  viandes, œufs, poissons et assimilés   charcut assimil      28725   
 1918  viandes, œufs, poissons et assimilés              oeuf      22507   
 2131        produits laitiers et assimilés    fromag assimil      12831   
 
                 alim_nom_fr  Kcal/100 g  Protéines/100 g  Glucides/100 g  \
 1571       lardon natur cru       270.0             16.6            1.01   
 1579         lardon fum cru       275.0             16.7            0.96   
 1580        lardon fum cuit       261.0             14.7            0.80   
 1918          omelet lardon       267.0             12.5            1.46   
 2131  pelardon fromag chevr       343.0             23.3            0.00   
 
       Lipides/100 g  Sucres/100 g  AG saturés/100 g  
 1571           22.1   

In [240]:
df_propre

,alim_grp_nom_fr,alim_ssgrp_nom_fr,alim_code,alim_nom_fr,Kcal/100 g,Protéines/100 g,Glucides/100 g,Lipides/100 g,Sucres/100 g,AG saturés/100 g
22,entrées et plats composés,soup,25900,soup lentill preemball a rechauff,55.0,3.74,6.60,1.12,0.70,0.45
23,entrées et plats composés,soup,25901,soup a volaill legum preemball a rechauff,30.8,3.09,2.60,0.80,1.20,0.30
24,entrées et plats composés,soup,25903,soup legum var preemball a rechauff,39.4,0.76,5.05,1.56,1.72,0.55
25,entrées et plats composés,soup,25904,soup poisson crustac preemball a rechauff,42.8,3.45,2.88,1.79,0.90,0.58
26,entrées et plats composés,soup,25905,soup legum var deshydrat reconstitu,34.5,0.86,4.43,1.22,1.07,0.39
...,...,...,...,...,...,...,...,...,...,...
3134,aides culinaires et ingrédients divers,ingredient div,20906,tofu soyeux preemball,54.3,5.00,1.53,2.90,1.13,0.47
3143,aides culinaires et ingrédients divers,ingredient div,31111,pollen frais,285.0,17.50,37.70,5.17,26.80,1.81
3144,aides culinaires et ingrédients divers,ingredient div,37000,bas pizz a crem,261.0,7.70,51.10,2.60,1.80,0.80
3145,aides culinaires et ingrédients divers,ingredient div,37002,bas pizz tomat,232.0,6.90,44.60,2.70,1.50,0.80


In [233]:
def food_matcher(nom_aliment):
    prod = normalize_text(nom_aliment)
    products_fit = words_in_column(df_propre, prod, 'alim_nom_fr')
    print(f"size products {products_fit.shape[0]}")
    shorter_description_sample = products_fit["alim_nom_fr"].apply(len).sort_values()[:10].index
    print(shorter_description_sample)
    ten_shorter_des_match = df_propre.loc[shorter_description_sample]
    print(ten_shorter_des_match["alim_ssgrp_nom_fr"].value_counts())
    best_cat = ten_shorter_des_match["alim_ssgrp_nom_fr"].value_counts().index[0]
    if best_cat == "nan":
        best_cat = ten_shorter_des_match["alim_ssgrp_nom_fr"].value_counts().index[1]
    ten_shorter_des_match = ten_shorter_des_match.loc[ten_shorter_des_match["alim_ssgrp_nom_fr"]==best_cat]
    best_bet = ten_shorter_des_match.iloc[0]
    print(f"Best match : {best_bet['alim_nom_fr']} \nBest cat : {best_cat}")
    return best_bet

In [234]:
def best_match(name):
    w_name = normalize_text(name)
    products_fit = words_in_column(df_propre, w_name, 'alim_nom_fr')
    print(products_fit)

In [235]:
def list_to_tab(list):
    tab = []
    print(list)
    for value in list:
        u_mes = value.split(' ', 1)[0]
        if (u_mes is None):
            u_mes = 1
        print(u_mes)
        attr = value.split(' ', 1)[1]
        temp = normalize_text(attr,False)
        print(temp)
        tab.append([u_mes, temp])
    print (tab)

In [236]:
list_to_tab(recipe)

["2\xa0c.à.s d'huile d'olive ", '10\xa0cl de vin rouge Sangiovese sec', '20\xa0cl de bouillon ', '500\xa0g de purée de tomate (passata)', "50\xa0g d'oignon haché", '50\xa0g de carotte jaune haché', ' poivre ', ' sel ', '500\xa0g de pâtes ', '300\xa0g de boeuf maigre haché', '100\xa0g de lard doux', '50\xa0g de céleri branche haché']
2 c.à.s
huile olive
10 cl
vin rouge Sangiovese sec
20 cl
bouillon
500 g
puree tomate passata
50 g
oignon hache
50 g
carotte jaune hache

poivre

sel
500 g
pates
300 g
boeuf maigre hache
100 g
lard doux
50 g
celeri branche hache
[['2\xa0c.à.s', 'huile olive'], ['10\xa0cl', 'vin rouge Sangiovese sec'], ['20\xa0cl', 'bouillon'], ['500\xa0g', 'puree tomate passata'], ['50\xa0g', 'oignon hache'], ['50\xa0g', 'carotte jaune hache'], ['', 'poivre'], ['', 'sel'], ['500\xa0g', 'pates'], ['300\xa0g', 'boeuf maigre hache'], ['100\xa0g', 'lard doux'], ['50\xa0g', 'celeri branche hache']]


In [237]:
df_propre.loc[df_propre["alim_nom_fr"].apply(lambda x: "lardon" in x)]

,alim_grp_nom_fr,alim_ssgrp_nom_fr,alim_code,alim_nom_fr,Kcal/100 g,Protéines/100 g,Glucides/100 g,Lipides/100 g,Sucres/100 g,AG saturés/100 g
1571,"viandes, œufs, poissons et assimilés",charcut assimil,28501,lardon natur cru,270.0,16.6,1.01,22.1,0.78,7.91
1579,"viandes, œufs, poissons et assimilés",charcut assimil,28720,lardon fum cru,275.0,16.7,0.96,22.6,0.80,8.00
1580,"viandes, œufs, poissons et assimilés",charcut assimil,28725,lardon fum cuit,261.0,14.7,0.80,22.1,0.80,8.25
1918,"viandes, œufs, poissons et assimilés",oeuf,22507,omelet lardon,267.0,12.5,1.46,23.5,0.68,9.41
2131,produits laitiers et assimilés,fromag assimil,12831,pelardon fromag chevr,343.0,23.3,0.00,27.8,0.00,18.60


In [238]:
food_matcher("lardon")

0       False
1       False
2       False
3       False
4       False
        ...  
1566    False
1567    False
1568    False
1569    False
1570    False
Length: 1571, dtype: bool
size products 0
Int64Index([], dtype='int64')
Series([], Name: alim_ssgrp_nom_fr, dtype: int64)


IndexError: index 0 is out of bounds for axis 0 with size 0

maintenir son poids 45 à 65% de glucides, 20 à 35% de lipides et 10 à 35% de protéines.

perdre du poids, un ratio de 35% de glucides, 30% de lipides et de 35% de protéines.

Par exemple, si vous décidez de suivre un régime cétogène pour maigrir, le ratio sera le suivant : 10% de glucides, 50% de lipides et 40% de protéines.

régime hyperprotéiné, le ratio sera de 10% de glucides, 25% de lipides et 65% de protéines.

masse musculaire 50% de glucides, 40% de protéines et 10% de lipides.